In [69]:
import torch 

What is going to be covered:
 1. Data Pre-processing
 2. Build Model
 3. Fit model to data(training)
 4. Making prediction and evaluating a model
 5. Saving & loading a model

In [70]:
# nn stands for neural network and this package contains the basic building blocks for creating neural networks in PyTorch
from torch import nn
import matplotlib.pyplot as plt

## Data (preparing and loading)
ML consists of two parts:
1. Feeding the model numerical data
2. Model learns patterns in that numerical representation

In [71]:
# Linear Regressor (mock)

# Known parameters
weight = 0.7
bias = 0.3

# Create
start = 0
end = 1
step = 0.02


X = torch.arange(start,end,step).unsqueeze(dim=1)
y = weight*X+bias

In [72]:
X

tensor([[0.0000],
        [0.0200],
        [0.0400],
        [0.0600],
        [0.0800],
        [0.1000],
        [0.1200],
        [0.1400],
        [0.1600],
        [0.1800],
        [0.2000],
        [0.2200],
        [0.2400],
        [0.2600],
        [0.2800],
        [0.3000],
        [0.3200],
        [0.3400],
        [0.3600],
        [0.3800],
        [0.4000],
        [0.4200],
        [0.4400],
        [0.4600],
        [0.4800],
        [0.5000],
        [0.5200],
        [0.5400],
        [0.5600],
        [0.5800],
        [0.6000],
        [0.6200],
        [0.6400],
        [0.6600],
        [0.6800],
        [0.7000],
        [0.7200],
        [0.7400],
        [0.7600],
        [0.7800],
        [0.8000],
        [0.8200],
        [0.8400],
        [0.8600],
        [0.8800],
        [0.9000],
        [0.9200],
        [0.9400],
        [0.9600],
        [0.9800]])

In [73]:
y

tensor([[0.3000],
        [0.3140],
        [0.3280],
        [0.3420],
        [0.3560],
        [0.3700],
        [0.3840],
        [0.3980],
        [0.4120],
        [0.4260],
        [0.4400],
        [0.4540],
        [0.4680],
        [0.4820],
        [0.4960],
        [0.5100],
        [0.5240],
        [0.5380],
        [0.5520],
        [0.5660],
        [0.5800],
        [0.5940],
        [0.6080],
        [0.6220],
        [0.6360],
        [0.6500],
        [0.6640],
        [0.6780],
        [0.6920],
        [0.7060],
        [0.7200],
        [0.7340],
        [0.7480],
        [0.7620],
        [0.7760],
        [0.7900],
        [0.8040],
        [0.8180],
        [0.8320],
        [0.8460],
        [0.8600],
        [0.8740],
        [0.8880],
        [0.9020],
        [0.9160],
        [0.9300],
        [0.9440],
        [0.9580],
        [0.9720],
        [0.9860]])

## Splitting data

In [74]:
# from sklearn.model_selection import train_test_split
# X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=1)
train_split = int(0.8 * len(X))
X_train, y_train = X[:train_split], y[:train_split]
X_test, y_test = X[train_split:], y[train_split:]

## Visualising

In [ ]:
def plot_predictions(train_data=X_train, 
                     train_labels=y_train, 
                     test_data=X_test, 
                     test_labels=y_test, 
                     predictions=None):
  """
  Plots training data, test data and compares predictions.
  """
  plt.figure(figsize=(10, 7))

  # Plot training data in blue
  plt.scatter(train_data, train_labels, c="b", s=4, label="Training data")
  
  # Plot test data in green
  plt.scatter(test_data, test_labels, c="g", s=4, label="Testing data")

  if predictions is not None:
    # Plot the predictions in red (predictions were made on the test data)
    plt.scatter(test_data, predictions, c="r", s=4, label="Predictions")

  # Show the legend
  plt.legend(prop={"size": 14});
plot_predictions()

## Build a model

In [ ]:
class LinearRegressorModel(nn.Module):
    def __init__(self):
        super().__init__()

        # weights and bias(model parameters) are the parameters of a neural network which will help in improving the model's accuracy of producing an output.
        self.weight =nn.Parameter(torch.randn(1,requires_grad=True,dtype=torch.float))
        self.bias= nn.Parameter(torch.randn(1,requires_grad=True,dtype=torch.float))

    # Computation for output
    def forward(self,x:torch.Tensor) -> torch.Tensor:
        return self.weight*x+self.bias
        

### Cheking the internals of the model

In [ ]:
torch.manual_seed(42)
model_0 = LinearRegressorModel()

# Check out the parameters
list(model_0.parameters())

In [ ]:
# list named parameters
model_0.state_dict()

### Make Predictions

In [ ]:
# When predicting/inferencing, the model need not keep track of gradient (which is done in training) -> .inference_mode() does that (stop tracking 
# gradients)
with torch.inference_mode():
    y_pred = model_0(X_test)
y_pred

In [ ]:
plot_predictions(predictions=y_pred)

# It can be noticed that the y_pred and y_test(actual) do  not overlap which says that the parameters of the model needs to updated.

## Train Model

For our model to update its parameters on its own, we'll need to add a few more things to our recipe.

And that's a loss function as well as an optimizer.

* **Loss Function**:Measures how wrong the model's predictions are. Lower the better.
* **Optimizer**:Takes into account the loss of the model. Adjust the parameters of the model (weights and bias)

In [ ]:
loss_fn = nn.L1Loss()
optimizer = torch.optim.SGD(model_0.parameters(),lr=0.01)

# lr => learning rate => determines how big the changes in the parameter should be.

In [ ]:
torch.manual_seed(42)

# Set the number of epochs (how many times the model will pass over the training data)
epochs = 100

# Create empty loss lists to track values
train_loss_values = []
test_loss_values = []
epoch_count = []

for epoch in range(epochs):
    ### Training

    # Put model in training mode (this is the default state of a model)
    model_0.train()

    # 1. Forward pass on train data using the forward() method inside 
    y_pred = model_0(X_train)

    # 2. Calculate the loss (how different are our models predictions to the ground truth)
    loss = loss_fn(y_pred, y_train)

    # 3. Zero grad of the optimizer
    optimizer.zero_grad()

    # 4. Loss backwards
    loss.backward()

    # 5. Progress the optimizer
    optimizer.step()

    ### Testing

    # Put the model in evaluation mode
    model_0.eval()

    with torch.inference_mode():
      # 1. Forward pass on test data
      test_pred = model_0(X_test)

      # 2. Caculate loss on test data
      test_loss = loss_fn(test_pred, y_test.type(torch.float)) # predictions come in torch.float datatype, so comparisons need to be done with tensors of the same type

      # Print out what's happening
      if epoch % 10 == 0:
            epoch_count.append(epoch)
            train_loss_values.append(loss.detach().numpy())
            test_loss_values.append(test_loss.detach().numpy())
            print(f"Epoch: {epoch} | MAE Train Loss: {loss} | MAE Test Loss: {test_loss} ")

### Visualise the results

In [ ]:
with torch.inference_mode():
    y_pred = model_0(X_test)
    plot_predictions(predictions=y_pred)

In [ ]:
y_test,y_pred

In [ ]:
weight,bias

In [ ]:
# Plot the loss curves
plt.plot(epoch_count, train_loss_values, label="Train loss")
plt.plot(epoch_count, test_loss_values, label="Test loss")
plt.title("Training and test loss curves")
plt.ylabel("Loss")
plt.xlabel("Epochs")
plt.legend();